# Naive Gauss Elimination

I will implement Naive Gauss Elimination from scrath to solve the following example. But notice that python has a simple function to solve linear equations.

\begin{align}
            2x_1 - x_2  - 2 x_3 &= 8 \\
            x_1 + 2x_2 +2x_3 &= 12 \\
            2x_1 + 4 x_2 -x_3 &= 4
        \end{align}

In [83]:
# Naive gauss elimination

A = [[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]]
b = [8,12, 4]

n = len(b)
x = [None] * n # create a list with n empty space to store the solution x

# forward elimination
for k in range(n-1):
    for i in range(k+1, n):
        factor = A[i][k]/A[k][k]
        for j in range(k, n): # the textbook say it is k+1 to n, which I think is not accurate
            A[i][j] = A[i][j] - factor * A[k][j]
        b[i] = b[i] - factor * b[k]
        
# backward substitution
x[n-1] = b[n-1]/A[n-1][n-1]
for i in range(n-2, -1, -1):
    sum = b[i]
    for j in range(i+1, n):
        sum = sum - A[i][j] * x[j]
        
    x[i] = sum/A[i][i]
    
print("The solutions of Ax = b: ", x)

The solutions of Ax = b:  [7.2, -1.6, 4.0]


# LU Decomposition and Solve for $Ax = b$

I will use LU Decomposition to solve the same example:

\begin{align}
            2x_1 - x_2  - 2 x_3 &= 8 \\
            x_1 + 2x_2 +2x_3 &= 12 \\
            2x_1 + 4 x_2 -x_3 &= 4
        \end{align}

In [176]:
import numpy as np

A = np.array([[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]])

b = np.array([8,12, 4])

def lu(A):
    
    #Get the number of rows
    A = A.astype('float64')
    n = A.shape[0]
    
    
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
            
        #Eliminate entries below i with row operations 
        #on U and reverse the row operations to 
        #manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U

def forward_substitution(L, b):
    
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x

def lu_solve(A, b):
    
    L, U = lu(A)
    
    y = forward_substitution(L, b)
    
    return back_substitution(U, y)

l, u = lu(A)
print("L matrix is: \n", l)
print("U matrix is: \n", u)

x = lu_solve(A,b)
print("The solution is: \n", x)

L matrix is: 
 [[1.  0.  0. ]
 [0.5 1.  0. ]
 [1.  2.  1. ]]
U matrix is: 
 [[ 2.  -1.  -2. ]
 [ 0.   2.5  3. ]
 [ 0.   0.  -5. ]]
The solution is: 
 [ 7.2 -1.6  4. ]
[[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 0.  -2.   1. ]]


# Matrix inverse

In [175]:
import numpy as np

A = np.array([[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]])

def matrix_inverse(A):
    # call lu decomposition to get L and U matrix
    L, U = lu(A) # this function is defined earlier. So make sure to run that cell first.
    n = len(A)
    A_inv = np.zeros((n,n))

    for i in range(n):
        unit_vector = np.zeros(n) 
        unit_vector[i] = 1 # try to create a vector with only ith element equals to 1
        A_inv[:,i] = back_substitution(U, forward_substitution(L, unit_vector)) # I combine forward and backward substitution to save space.
    return A_inv

A_inv =  matrix_inverse(A)
print("The inverse of matrix A is: \n", A_inv)

The inverse of matrix A is: 
 [[ 0.4   0.36 -0.08]
 [-0.2  -0.08  0.24]
 [-0.    0.4  -0.2 ]]


# System Condition Examples

In [126]:
# Scale the matrix of coefficients, $A$, so that the largest element in each row is 1.
A = np.array([[1, 1, 1], 
     [1, 0.5, 0.5],
     [1, 1, 0.999]])
print("The matrix A is: \n", A)

# If there are elements of $A^{-1}$ that are several orders of magnitude greater than one, it is likely that the system is ill-conditioned.
A_inv =  matrix_inverse(A)
print("The inverse of matrix A is: \n", A_inv)


# Multiply the inverse by the original coefficient matrix and assess whether the result is close to the identity matrix. If not, it indicates ill conditioning.
A_A_inv = np.dot(A, A_inv)
print("The A times A_inverse is: \n", A_A_inv)

# Invert the inverted matrix and assess whether the result is sufficiently close to the original coefficient matrix. If not, it again indicates that the system is ill-conditioned.
A_inv_inv = matrix_inverse(A_inv)
print("The inverse of inverse A is: \n", A_inv_inv)


The matrix A is: 
 [[1.    1.    1.   ]
 [1.    0.5   0.5  ]
 [1.    1.    0.999]]
The inverse of matrix A is: 
 [[   -1.     2.     0.]
 [ -998.    -2.  1000.]
 [ 1000.    -0. -1000.]]
The A times A_inverse is: 
 [[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [2.13162821e-17 0.00000000e+00 1.00000000e+00]]
The inverse of inverse A is: 
 [[1.    1.    1.   ]
 [1.    0.5   0.5  ]
 [1.    1.    0.999]]


# Gauss-Seidel Method

I will use Gauss-Seidel Method to solve the same example. The result is divergence!

\begin{align}
            2x_1 - x_2  - 2 x_3 &= 8 \\
            x_1 + 2x_2 +2x_3 &= 12 \\
            2x_1 + 4 x_2 -x_3 &= 4
        \end{align}

In [173]:
# Gauss-Seidel Method:

import numpy as np
A = [[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]]
b = [8,12, 4]

n = len(b)
x = np.array([0] * n) # create a list with n empty space to store the solution x

e_threshold = 0.001 # relative error tolerance 0.1% 
e_rel = 1
counter = 0
# keep looping until the stopping criteria is met: e_rel < e_threshold or counter >= 1000 
while (e_rel > e_threshold and counter < 100):
    counter = counter + 1
    print("Iteration number: ", counter)
    x_old = x.copy() # You will have problems if you use x_old = x: because x_old will keep changing with x, since x is a list. Therefore, we need to create a copy of x and save it to x_old 
    for i in range(n):
        sum = b[i]
        for j in range(n):
            if (j != i):
                sum = sum - A[i][j]* x[j]        
        x[i] = sum/A[i][i]
        
    x_new = x.copy()


    e_rel = max(abs(np.divide((x_new - x_old), x_new)))
    
print("The solutions of Ax = b: ", x)

Iteration number:  1
Iteration number:  2
Iteration number:  3
Iteration number:  4
Iteration number:  5
Iteration number:  6
Iteration number:  7
Iteration number:  8
Iteration number:  9
Iteration number:  10
Iteration number:  11
Iteration number:  12
Iteration number:  13
Iteration number:  14
Iteration number:  15
Iteration number:  16
Iteration number:  17
Iteration number:  18
Iteration number:  19
Iteration number:  20
Iteration number:  21
Iteration number:  22
Iteration number:  23
Iteration number:  24
Iteration number:  25
Iteration number:  26
Iteration number:  27
Iteration number:  28
Iteration number:  29
Iteration number:  30
Iteration number:  31
Iteration number:  32
Iteration number:  33
Iteration number:  34
Iteration number:  35
Iteration number:  36
Iteration number:  37
Iteration number:  38
Iteration number:  39
Iteration number:  40
Iteration number:  41
Iteration number:  42
Iteration number:  43
Iteration number:  44
Iteration number:  45
Iteration number:  

<ipython-input-173-f68a246e0600>:24: RuntimeWarning: overflow encountered in long_scalars
  sum = sum - A[i][j]* x[j]


In [170]:
max(abs(np.divide((x_new - x_old), x_new)))
x_new
x_old

array([2, 1, 2])

# Python has some nice functions. 

Just for your information in the future. Please do not directly use these functions for homework, otherwise it is not interesting any more.

In [77]:
# use python functions to get pivoted LU decomposition, this will be different from naive LU decomposition
from scipy.linalg import lu
import numpy as np

A = [[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]]

p, l, u = lu(A, permute_l = False) # pivoted LU decomposition, so L = pl
print("P matrix is: \n", p)
print("L matrix is: \n", l)
print("U matrix is: \n", u)


P matrix is: 
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
L matrix is: 
 [[1.  0.  0. ]
 [1.  1.  0. ]
 [0.5 0.5 1. ]]
U matrix is: 
 [[ 2.  -1.  -2. ]
 [ 0.   5.   1. ]
 [ 0.   0.   2.5]]


In [45]:
# use python functions to solve Ax = b
import numpy as np
A = [[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]]
b = [8,12, 4]

x = np.dot(np.linalg.inv(A), b)
print("The solutions of Ax = b: ", x)

The solutions of Ax = b:  [ 7.2 -1.6  4. ]


In [84]:
# use python functions to calculate inverse of A
import numpy as np
A = [[2, -1, -2], 
     [1, 2, 2],
     [2, 4, -1]]

A_inv = np.linalg.inv(A)
print("The inverse of A is : \n", A_inv )

The inverse of A is : 
 [[ 0.4   0.36 -0.08]
 [-0.2  -0.08  0.24]
 [ 0.    0.4  -0.2 ]]
